<img src=https://brand.uark.edu/_resources/images/UA_Logo_Horizontal.jpg width="400" height="96">

###_Biomedical Image Analysis & Artificial Intelligence_

# Notebook 3.2 Introduction to Deep Learning
---
##### The purpose of this notebook is to cover the basics of deep learning.



### Required packages
---
##### **_NOTE: This notebook is accelerated with the use of GPU hardware, but is not required. please refer to notebook 2.4_ParallelProcessing if you need a refresher on how to do this._**
##### **_Run this code chunk first. If you encounter an error when trying to run code chunks in this notebook, then first try re-running this chunk._**


In [ ]:
# Import all of the necessary packages
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as T
from tqdm.notebook import tqdm
from IPython.display import clear_output

# Training a model with Deep Learning
---
##### Training some type of AI model with deep learning is relatively straightforward, but does require a few mathematical tricks. The tricks used in this notebook will not be covered in detail due to being out of the scope of this module, however, the terms used in this notebook can easily be searched to learn more.
##### Training a deep learning model can be broken up into 4 steps, and we will go through each of these steps in this notebook:
1. Forward pass
2. Loss calculation
3. Back-propogation
4. Gradient descent

##### To walk through each step, we will use a common example of a deep learning model, in this case a multi-layer perceptron (_MLP_).  The input is a small image of a handwritten digit and the model predicts what handwriten digit is shown in the image. Here are some of the images that will be used:
<img src="https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png">


# Step 0: Network Initialization
---
##### Before any network training is done, we must first inspect the dataset, and decide on how our model will be deisgned. 
##### Each image is 28 by 28 pixels (784 pixels total), which we can treat as our number of _inputs_. The handwritten digits range from 0-9, meaning there are 10 possible digits that we would want to predict, which is the number of _outputs_ that we want. Finally, the rest of the network architecture (the number of layers and number of perceptrons per layer) is very arbitrary, and finding the ideal network attributes ultimately relies on trial and error. For this particular example, we will use 512, 256 and 128 perceptrons for the first, second, and third layer, respectively.

<img src="https://github.com//aewoessn/outreach-program-2021/blob/main/figures/outreach_mlp.png?raw=true" alt="outreach_mlp.png" width="469.5" height="488">

##### The `torch` package we are using is made for deep learning neural networks.  It comes with convenience functions so we do not have to manually create the network.  As a result, the  code chunk below looks a little different from previous examples. Within the `torch` environment, _layers_ are defined which contain all of the perceptron weights for a layer, which is followed by a layer that specifies the _activation function_. In this example, the _activation function_ is different than what was previous discussed due to specific mathematical requirements. If you are interested, a list of common activation functions, along with how they look plotted, can be found [here](https://ml-cheatsheet.readthedocs.io/en/latest/activation_functions.html).

In [ ]:
# Define network architecture
number_of_inputs = 784
number_of_layers = 3
number_of_perceptrons_per_layer = [512,256,128]
number_of_outputs = 10

# Generate the network (this will look different than the previous notebook, but is doing the same exact thing)
test_model = nn.Sequential(
    nn.Flatten(start_dim=1, end_dim=-1),
    nn.Linear(number_of_inputs,number_of_perceptrons_per_layer[0]),
    nn.ReLU(),
    nn.Linear(number_of_perceptrons_per_layer[0],number_of_perceptrons_per_layer[1]),
    nn.ReLU(),
    nn.Linear(number_of_perceptrons_per_layer[1],number_of_perceptrons_per_layer[2]),
    nn.ReLU(),
    nn.Linear(number_of_perceptrons_per_layer[2],number_of_outputs)
)

# Print out message indicating that the network has been created
print('The MLP model has successfully been generated!')

### Loading the dataset
---
##### The following code chunk will download the MNIST dataset, but will not store it on your computer. Please note that this may take ~5 minutes to download.

In [ ]:
# Load the MNIST digits dataset
import os

# Check to see if the training file exists...
filename = "/content/MNIST/processed/training.pt"
if not os.path.isfile(filename):
  # ...and if it does not, then download it,...
  !gdown --id 1jNqLo0LBYrPcGzaDBBA60sqGlclS_nX_

  # ...check to see if the destination path exists,...
  if not os.path.isdir("/content/MNIST/processed/"):
    os.makedirs("/content/MNIST/processed/")

  # ...and move the file.
  os.rename("/content/training.pt",filename)

# Check to see if the test file exists...
filename = "/content/MNIST/processed/test.pt"
if not os.path.isfile(filename):
  # ...and ff it does not, then download it,...
  !gdown --id 178t2_ul2VvnAHuClPhlE27qHkpE5I8S1

  # ...and move the file.
  os.rename("/content/test.pt",filename)

# Create the dataset
mnist_data = torchvision.datasets.MNIST(root='/content/', transform=T.ToTensor())
clear_output()

# Step 1: Forward pass
---
##### The _forward pass_ step was actually described in the previous notebook as just a weighted summation of all inputs into a single perceptron, and is done for the entire network. Since 10 outputs were specified in the beginning, the outcome of the _forward pass_ is going to be 10 _probabilities_, which are the likelihood that the model thinks that the input image corresponds to every possible digit.
##### The output of this code chunk shows the input image, as well as the probability, or percent confidence, that the model is confident that the input image is each digit.  As you can see, an untrained network is not going to know what digit it is looking at.

In [ ]:
# Get the first image in the MNIST dataset
input_image = mnist_data.data[0,:,:].unsqueeze(0).unsqueeze(0).float() / 255.

# Compute the forward pass of the image
output = test_model.forward(input_image)

# Get the probabilities by passing the output through a softmax
prob = torch.softmax(output,1)*100

# Show the image
fig = plt.Figure()
img = plt.imshow(input_image.squeeze(0).squeeze(0))
ax = plt.gca()
ax.set_title('Input image',fontsize='x-large');
plt.show()

# Generate a report
print('\nNumber - Probability [%]\n')
for i in range(10):
  print(str(i) + '   -    ' + str(prob[0,i].detach().numpy()) + '\n')

# Step 2: Loss calculation
---
##### When the network is finished computing the forward pass from an image, we need to see how well or how poor it was at predicting the correct digit. To do this, we use a _loss function_, which penalizes the network based on how off the prediction was.
##### For this particular application, we are really assessing if the input image was predicted to be in the correct group or _class_, therefore it is a _classification_ network. Alternatively, a network could also output some meaningful number or numbers, which is called a _regression_ network.
##### The loss function that we are going to use for this example is called a _cross-entropy loss_. 

In [ ]:
# Define which loss function we will be using
loss = nn.CrossEntropyLoss()

# Get the correct label for the image
input_label = mnist_data.targets[0].unsqueeze(0)

# Calculate the loss based on the output from the model
model_loss = loss(output,input_label)
print("The numerical loss value is: " + str(model_loss.detach().numpy()) + ". The lower this value, the better the model is at predicting the correct digit.")

# Step 3: Back-propogation
---
##### After a loss value has been calculated, we need to adjust the all of the weights within the network so that it can do better later on. The first thing we need to do is calculate how drastically we need to adjust each weight in the network. There is a lot of calculus involved with this step, and this module will not go over these details. However, you should know that this step is called _back-propogation_, since the magnitude of how drastically the weights should be adjusted are propogated backwards through the network, starting with the loss value (output) and ending with the input weights.

In [ ]:
# Back-propogate the loss function through the network
model_loss.backward()
print('The loss value has been propogated to each weight. Now each weight can be adjusted')

# Step 4: Gradient descent
---
##### Now that we have calculated how drastically we want to adjust each weight, we can actually adjust them using what is called a _gradient descent_ algorithm. Gradient descent is a way to step through a function to end up at some area that is a minimum value with respect to the values around it.
##### At each step, the amount that we adjust the weight is usually orders of magnitude greater that the amount that we actually want to adjust the weights. To scale the amount that we change the weight, we use a very small _learning rate_.
##### A good analogy for gradient descent is to picture a mathematical function, and imagine you place a marble somewhere on the surface. When you let go of the marble, gravity will move the marble to the lowest spot on the function.
##### Below is a code chunk that demonstrates gradient descent.

In [ ]:
#@title --- Hidden code (double-click to show code) ---
# Use y = x^2 as an example for gradient descent
input = np.arange(-3,3,0.001)
output_func = lambda x: x**2
output = input**2

# Pick a starting point for the 'marble'
starting_point = -2.7
current_point = starting_point
current_slope = (output_func(current_point-0.1) - output_func(current_point+0.1)) / ((current_point-0.1)-(current_point+0.1))

fig = plt.figure(figsize=(20,10))
func_line = plt.plot(input,output,zorder=1)
point_plot = plt.plot(current_point,output_func(current_point),'ro',markersize=12)
ax = plt.gca()
ax.set_title("Current slope: " + str(current_slope))
ax.set_ylabel('Y = X^2',fontsize='x-large')
plt.show()

tol = 1e-4
learning_rate = 0.01
momentum = 0.9
velocity = 0
count = 0
while np.abs(current_slope) > tol:
  clear_output(wait=True)

  velocity = ((velocity*momentum) - (learning_rate*current_slope))
  current_point += velocity 
  current_slope = (output_func(current_point-0.1) - output_func(current_point+0.1)) / ((current_point-0.1)-(current_point+0.1))

  fig = plt.figure(figsize=(20,10))
  ax = plt.gca()
  func_line = plt.plot(input,output,zorder=1);
  point_plot = plt.plot(current_point,output_func(current_point),'ro',markersize=12);

  if np.abs(current_slope) > tol:
    ax.set_title("Current slope: " + str(current_slope))
    ax.set_ylabel('Y = X^2',fontsize='x-large')
  else:
    ax.set_title("Done after " + str(count) + " iterations.")
    ax.set_ylabel('Y = X^2',fontsize='x-large')
    
  plt.show()
  count += 1

### Implementing Gradient Descent
---
##### There are few different gradient descent algorithms, but they essentially achieve the same goal. Here is an animation showing a few common algorithms:
<img src="https://github.com/aewoessn/outreach-program-2021/blob/main/figures/Optimizer.gif?raw=true" alt="Optimizer.gif" width="400" height="320"><img src="https://github.com/aewoessn/outreach-program-2021/blob/main/figures/OptimizerLegend.png?raw=true" alt="OptimizerLegend.png" width="163" height="73">

##### Additionally, weights are randomly generated during network initialization because there could potentially be multiple _local minima_ for a single loss function, where some are better than others. Here is an example of multiple different starting locations all with the same optimizer that end up in different _local minima_.
<img src="https://github.com/aewoessn/outreach-program-2021/blob/main/figures/LocalMinima.gif?raw=true" alt="LocalMinima.gif" width="400" height="320">

##### Similar to the back-propogation, the `torch` package provides very convenient functions to accomplish this step, as seen in the following code chunk.

In [ ]:
# Specify the optimizer (or gradient descent algorithm)
optimizer = torch.optim.Adam(test_model.parameters())

# Gradient descent iteration
optimizer.step()
print('The network weights have been adjusted for the input image.')

### Re-evaluating the model
---
##### To double check that the model was successfully trained on the image, we can re-evaluate the same image as before.
##### The output of this code chunk will show the input image, as well as the confidence of the model for the digit in the image _after backpropogation_.
##### The model should be slightly more confident in the correct answer now since we trained the model on the input image.

In [ ]:
# Compute the forward pass of the image
output = test_model.forward(input_image)

# Get the probabilities by passing the output through a softmax
prob = torch.softmax(output,1)*100

# Show the image
fig = plt.Figure()
img = plt.imshow(input_image.squeeze(0).squeeze(0))
ax = plt.gca()
ax.set_title('Input image',fontsize='x-large');
plt.show()

# Generate a report
print('\nNumber - Probability [%]\n')
for i in range(10):
  print(str(i) + '   -    ' + str(prob[0,i].detach().numpy()) + '\n')

# Training the model on the whole dataset
---
##### Now that we have run through a single iteration of training for a single image, we can expand this to train the model over the entire dataset. Training on different digit images will change the weights that we just updated, but over time, the average loss for each digit will decrease, resulting in a model that can correctly predict every digit. 
##### When training over the entire dataset, the model is said to have completed an _epoch_, or lifetime of the dataset. At this point, we re-collect all of the images used for training, and continue to train the model. We can do this for as long as we would like, but **_deep learning models are usually benchmarked to each other based on the accuracy of the model as well as how long it took to train the model._** This concept is good to keep in mind, and will be covered in detail in a few notebooks. For now, we are only interested on training a network with the entire dataset.
##### Additionally, training for a very large amount of epochs (>100) can result in diminishing returns or even negatively impact the model.
##### In the below code chunk, we combine all of the deep learning steps and train the MLP model end-to-end. **_Whether you use a GPU or not, this should only take a few minutes._**

In [ ]:
#@title --- Hidden code (double-click to show code) ---

# Give some context of what is going on
print('Preparing the network and dataset...')

# Define network architecture
number_of_inputs = 784
number_of_layers = 3
number_of_perceptrons_per_layer = [512,256,128]
number_of_outputs = 10

# Generate the network (this will look different than the previous notebook, but is doing the same exact thing)
model = nn.Sequential(
    nn.Flatten(start_dim=1, end_dim=-1),
    nn.Linear(number_of_inputs,number_of_perceptrons_per_layer[0]),
    nn.ReLU(),
    nn.Linear(number_of_perceptrons_per_layer[0],number_of_perceptrons_per_layer[1]),
    nn.ReLU(),
    nn.Linear(number_of_perceptrons_per_layer[1],number_of_perceptrons_per_layer[2]),
    nn.ReLU(),
    nn.Linear(number_of_perceptrons_per_layer[2],number_of_outputs)
)

try:
  model = model.cuda()
  canUseGPU = True
except:
  print('Warning: The GPU has not been enabled. Training may take longer than expected.')
  canUseGPU = False

# Load the MNIST digits dataset
batchsize = 10
data_loader = torch.utils.data.DataLoader(mnist_data,batch_size=batchsize,shuffle=True)

# Specify the number of epochs
number_of_epochs = 3

# Pull out a small subset of the dataset that will be used to benchmarking purposes
test_images = mnist_data.data[[1,3,5,7,9,0,13,15,17,4],:,:].float().unsqueeze(1) / 255.

if canUseGPU:
  test_images = test_images.cuda()

with torch.no_grad():
  test_output = model.forward(test_images).cpu()
  test_prob = torch.transpose(torch.softmax(test_output,1)*100,0,1)

print('...Done')

# Plot the results
fig = plt.figure(figsize=(10,5))
plt.imshow(test_prob)
ax = plt.gca()
plt.xticks([])
secax = ax.secondary_xaxis('top')
secax.set_xticks(np.arange(0,10))
secax.set_xlabel('Predicted digit',fontsize='x-large')
plt.yticks(np.arange(0,10))
ax.set_ylabel('Actual digit',fontsize='x-large')
plt.clim(0,100)
cbar = plt.colorbar()
cbar.set_label('Confidence in prediction [%]',fontsize='x-large')
plt.title('Output from epoch: 0 of ' + str(number_of_epochs),fontsize='x-large')
plt.show()

# Define which loss function we will be using
loss = nn.CrossEntropyLoss()

# Specify the optimizer (or gradient descent algorithm)
optimizer = torch.optim.Adam(model.parameters())

# Go through each epoch
for ep in range(number_of_epochs):

  # Set up a progress bar for training
  with tqdm(total=len(mnist_data), desc=f'Epoch {ep + 1}/{number_of_epochs}', unit='img') as pbar:
    
    # Go through all of the images in the dataset
    for i, data in enumerate(data_loader, 0):
      # Get the current minibatch
      inputs, labels = data

      if canUseGPU:
        inputs = inputs.cuda()
        labels = labels.cuda()

      # Clear gradients (this is just required)
      optimizer.zero_grad()

      # Perform forward pass
      output = model.forward(inputs)

      # Calculate loss
      loss_value = loss(output,labels)

      # Back-propogate
      loss_value.backward()

      # Gradient descent
      optimizer.step()  

      # Update progress bar
      pbar.update(batchsize)

  # Clear the output
  clear_output(wait=True)

  with torch.no_grad():
    test_output = model.forward(test_images).cpu()
    test_prob = torch.softmax(test_output,1)*100

  # Plot the results
  fig = plt.figure(figsize=(10,5))
  plt.imshow(test_prob)
  ax = plt.gca()
  plt.xticks([])
  secax = ax.secondary_xaxis('top')
  secax.set_xticks(np.arange(0,10))
  secax.set_xlabel('Predicted digit',fontsize='x-large')
  plt.yticks(np.arange(0,10))
  ax.set_ylabel('Actual digit',fontsize='x-large')
  plt.clim(0,100)
  cbar = plt.colorbar()
  cbar.set_label('Confidence in prediction [%]',fontsize='x-large')
  plt.title('Output from epoch: '+ str(ep+1) + ' of ' + str(number_of_epochs),fontsize='x-large')
  plt.show()

# Show each digit in the small test batch and display the digit predicted with the models confidence
f = plt.figure(figsize=(15,15))

for i in range(10):
  f.add_subplot(2,5,i+1)
  plt.imshow(test_images[i,:,:].squeeze(0).cpu())
  plt.gca().set_title("Predicted: " + str(torch.argmax(test_prob[i,:]).detach().numpy()) + " (" + str(torch.max(test_prob[i,:]).detach().numpy()) + "%)")
f.subplots_adjust(bottom=0.55)

# Trained model analysis
---
##### To motivate the next topic, we can map the average weight for each pixel in the input image, as shown in the following code chunk.

In [ ]:
# Pull out the first set of weights
count = 0
for params in model.parameters():
  if count == 0:
    weight = params.cpu().detach()
  count += 1

# Get the average weight for each pixel in the input image
sum_weight = torch.sum(torch.abs(weight),dim=0)
img_weight = torch.reshape(sum_weight,(28,28))

# Display the image
plt.figure(figsize=(10,10))
plt.imshow(img_weight);
plt.gca().set_title('Summed weights for each pixel of the input image')
cbar = plt.colorbar(shrink=0.8);
cbar.set_label('Sum of all weights at each pixel',fontsize='x-large')

### Interpreting results
---
##### By looking at a map of the average weights for the input image, we see that most of the pixels toward the center of the input image are non-zero. More interestingly, the pixels toward the corners of the input image are very close to zero.
##### Suppose some of our digits are not centered in the image... 
1. Do you think we would get different results? 
2. How do you think we can get around this issue?

# Finishing up
---
##### **_If you used a GPU and are fininshed with the notebook, please make sure to end your session with Google Colab by selecting:_**
> ### Runtime > Manage sessions
##### **_A window will pop up and you need to locate the current notebook and select:_**
> ### TERMINATE

# Ready for the next notebook?
---
##### You can click [here](https://colab.research.google.com/github/aewoessn/outreach-program-2021/blob/main/notebooks/3.3_ConvolutionalNeuralNetworks.ipynb) to take you to the next notebook.